In [ ]:
from tensorflow import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from keras.models import Sequential
from keras.layers.convolutional import ZeroPadding2D
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
!git clone "https://github.com/moheb432/principles-of-smile.git"

Cloning into 'principles-of-smile'...
remote: Enumerating objects: 6505, done.
remote: Counting objects: 100% (1726/1726), done.
remote: Compressing objects: 100% (1711/1711), done.
remote: Total 6505 (delta 31), reused 1704 (delta 15), pack-reused 4779
Receiving objects: 100% (6505/6505), 341.16 MiB | 21.12 MiB/s, done.
Resolving deltas: 100% (294/294), done.
Checking out files: 100% (6999/6999), done.


In [ ]:
df = pd.read_csv("/content/principles-of-smile/dataset_and_augmented.csv")
df

,img,9-gummy,16-Incisal Embrasure,4-color,41-Central Incisor W/H Ratio,18-black triangle,8-gaps,52-Crooked
0,41001_B.jpg,1,1,1,1,0.0,0,0
1,41002_B.jpg,0,1,1,1,0.0,1,0
2,41003_B.jpg,1,1,0,1,0.0,0,0
3,41004_B.jpg,1,1,1,1,0.0,1,0
4,41005_B.jpg,1,1,0,1,0.0,0,0
...,...,...,...,...,...,...,...,...
2458,aug_(f15)_0_6500.jpg,0,1,1,1,1.0,1,1
2459,aug_(f15)_0_8219.jpg,0,1,1,1,1.0,1,1
2460,aug_(f15-a)_0_4936.jpg,0,1,1,0,0.0,0,0
2461,aug_(f15-a)_0_7990.jpg,0,1,1,0,0.0,0,0


In [ ]:
missing = df.isnull().sum()
missing


img                             0
9-gummy                         0
16-Incisal Embrasure            0
4-color                         0
41-Central Incisor W/H Ratio    0
18-black triangle               8
8-gaps                          0
52-Crooked                      0
dtype: int64

In [ ]:
classes = ['9-gummy','16-Incisal Embrasure','4-color','41-Central Incisor W/H Ratio','18-black triangle','8-gaps','52-Crooked']


In [ ]:
def append_ext(fn):
    return str(fn)+".jpg"

#loading dataset labels from csv 
df = pd.read_csv("/content/principles-of-smile/dataset_and_augmented.csv")

# df["img"] = df["img"].apply(append_ext)

df = df.replace(np.nan, 0)
classes = ['9-gummy', '16-Incisal Embrasure', '4-color', '41-Central Incisor W/H Ratio','18-black triangle','8-gaps', '52-Crooked']
df

,img,9-gummy,16-Incisal Embrasure,4-color,41-Central Incisor W/H Ratio,18-black triangle,8-gaps,52-Crooked
0,41001_B.jpg,1,1,1,1,0.0,0,0
1,41002_B.jpg,0,1,1,1,0.0,1,0
2,41003_B.jpg,1,1,0,1,0.0,0,0
3,41004_B.jpg,1,1,1,1,0.0,1,0
4,41005_B.jpg,1,1,0,1,0.0,0,0
...,...,...,...,...,...,...,...,...
2458,aug_(f15)_0_6500.jpg,0,1,1,1,1.0,1,1
2459,aug_(f15)_0_8219.jpg,0,1,1,1,1.0,1,1
2460,aug_(f15-a)_0_4936.jpg,0,1,1,0,0.0,0,0
2461,aug_(f15-a)_0_7990.jpg,0,1,1,0,0.0,0,0


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es=EarlyStopping(monitor='val_binary_accuracy',patience=10,mode='max',restore_best_weights=True)

In [ ]:
def MODEL():
  model = Sequential()
  model.add(keras.Input(shape=(224,224,3)))
  model.add(Conv2D(64, (3, 3), activation='relu',name="conv1_1"))
  model.add(Conv2D(64, (3, 3), activation='relu',name="conv1_2"))
  model.add(MaxPool2D((2,2), strides=(2,2)))

  model.add(Conv2D(128, (3, 3), activation='relu',name="conv2_1"))
  model.add(Conv2D(128, (3, 3), activation='relu',name="conv2_2"))
  model.add(MaxPool2D((2,2), strides=(2,2)))

  model.add(Conv2D(256, (3, 3), activation='relu',name="conv3_1"))
  model.add(Conv2D(256, (3, 3), activation='relu',name="conv3_2"))
  model.add(Conv2D(256, (3, 3), activation='relu',name="conv3_3"))
  model.add(MaxPool2D((2,2), strides=(2,2)))
  
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv4_1"))
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv4_2"))
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv4_3"))
  model.add(MaxPool2D((2,2), strides=(2,2)))

  model.add(Conv2D(512, (3, 3), activation='relu',name="conv5_1"))
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv5_2"))
  model.add(Conv2D(512, (3, 3), activation='relu',name="conv5_3"))
  model.add(MaxPool2D((2,2), strides=(2,2)))

  model.load_weights('/content/drive/MyDrive/7classes_lr0001/7classes_b16_lr0002.h5', by_name=True,skip_mismatch=True)

  #freezing layers
  for layer in model.layers:
      layer.trainable = False

  model.get_layer("conv3_1").trainable =True
  #model.get_layer("conv3_3").trainable =True
  #model.get_layer("conv4_1").trainable =True
  model.get_layer("conv4_1").trainable =True
  model.get_layer("conv4_2").trainable =True
  model.get_layer("conv5_3").trainable =True
  model.get_layer("conv5_1").trainable =True


  #adding output layers
  model.add(Flatten())

  model.add( Dense(64,use_bias=False, kernel_regularizer=keras.regularizers.L2(0.02)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))

  model.add(Dropout(0.5))

  model.add( Dense(16, use_bias=False,kernel_regularizer=keras.regularizers.L2(0.015)))
  model.add(BatchNormalization())
  model.add(Activation("relu"))

  model.add(Dense(7, activation='sigmoid'))
  # summarize
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss = 'BinaryCrossentropy',metrics=['binary_accuracy'])

  # model.summary()
  return model

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
i=1
df_metrics = pd.DataFrame()
kf = KFold(n_splits = 5, shuffle = True, random_state = None)
for train_index, test_index in kf.split(df):
    trainData = df.iloc[train_index]
    testData = df.iloc[test_index]
    print('Initializing Kfold %s'%str(i))
    print('Train shape:',trainData.shape)
    print('Test shape:',testData.shape)
    epochs = 30
    train_datagen = ImageDataGenerator(samplewise_center=True,rescale=1./255,validation_split=0.2)
    test_datagen = ImageDataGenerator(rescale=1. / 255) 

    training_set = train_datagen.flow_from_dataframe(dataframe=trainData,
                                                            directory="/content/principles-of-smile/K-fold/train",
                                                            x_col="img",
                                                            y_col=classes,
                                                            subset="training",
                                                            target_size=(224, 224),
                                                            batch_size=16,
                                                            class_mode="raw",seed=0)


    # valid_data_generation = ImageDataGenerator(samplewise_center=True,rescale = 2/255.)

    valid_set = train_datagen.flow_from_dataframe(dataframe=trainData,
                                                            directory="/content/principles-of-smile/K-fold/train",
                                                            x_col="img",
                                                            y_col=classes,
                                                            subset="validation",
                                                            target_size=(224, 224),
                                                            batch_size=16,
                                                            class_mode="raw",seed=0)

    # preprocessing the testing set
    # test_data_generation = ImageDataGenerator(samplewise_center=True,rescale = 2/255.)

    test_set = test_datagen.flow_from_dataframe(dataframe=testData,
                                                            directory="/content/principles-of-smile/K-fold/test",
                                                            x_col="img",
                                                            y_col=classes,
                                                            target_size=(224, 224),
                                                            batch_size=1,
                                                            class_mode="raw",seed=0, shuffle = False)
    initial_learning_rate = 0.0002
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate),loss = 'BinaryCrossentropy',metrics=['binary_accuracy'])
    #fit model
    history=model.fit(x=training_set, validation_data=valid_set, epochs=50 ,callbacks=[es])
    i+=1

Initializing Kfold 1
Train shape: (1970, 8)
Test shape: (493, 8)
Found 330 validated image filenames.
Found 82 validated image filenames.
Found 14 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1558 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 479 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Epoch 1/50
21/21 [==============================] - 36s 763ms/step - loss: 3.2184 - binary_accuracy: 0.5255 - val_loss: 3.0786 - val_binary_accuracy: 0.6620
Epoch 2/50
21/21 [==============================] - 6s 295ms/step - loss: 2.9372 - binary_accuracy: 0.6104 - val_loss: 2.8503 - val_binary_accuracy: 0.6829
Epoch 3/50
21/21 [==============================] - 6s 295ms/step - loss: 2.7086 - binary_accuracy: 0.6329 - val_loss: 2.6417 - val_binary_accuracy: 0.6829
Epoch 4/50
21/21 [==============================] - 6s 296ms/step - loss: 2.4964 - binary_accuracy: 0.6558 - val_loss: 2.4500 - val_binary_accuracy: 0.6829
Epoch 5/50
21/21 [==============================] - 7s 352ms/step - loss: 2.3089 - binary_accuracy: 0.6654 - val_loss: 2.2777 - val_binary_accuracy: 0.6864
Epoch 6/50
21/21 [==============================] - 6s 299ms/step - loss: 2.1324 - binary_accuracy: 0.6805 - val_loss: 2.1225 - val_binary_accuracy: 0.7213
Epoch 7/50
21/21 [==============================] - 6s 291ms/st

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1561 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 477 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Epoch 1/50
21/21 [==============================] - 15s 649ms/step - loss: 1.9681 - binary_accuracy: 0.6773 - val_loss: 1.9525 - val_binary_accuracy: 0.6755
Epoch 2/50
21/21 [==============================] - 6s 297ms/step - loss: 1.8013 - binary_accuracy: 0.6890 - val_loss: 1.8030 - val_binary_accuracy: 0.6790
Epoch 3/50
21/21 [==============================] - 6s 295ms/step - loss: 1.6573 - binary_accuracy: 0.6803 - val_loss: 1.6728 - val_binary_accuracy: 0.6966
Epoch 4/50
21/21 [==============================] - 6s 293ms/step - loss: 1.5301 - binary_accuracy: 0.6969 - val_loss: 1.5556 - val_binary_accuracy: 0.6755
Epoch 5/50
21/21 [==============================] - 6s 290ms/step - loss: 1.4096 - binary_accuracy: 0.7173 - val_loss: 1.4634 - val_binary_accuracy: 0.7055
Epoch 6/50
21/21 [==============================] - 6s 296ms/step - loss: 1.3210 - binary_accuracy: 0.7143 - val_loss: 1.3651 - val_binary_accuracy: 0.6437
Epoch 7/50
21/21 [==============================] - 6s 289ms/st

...

21/21 [==============================] - 8s 322ms/step - loss: 1.3285 - binary_accuracy: 0.6948 - val_loss: 1.3572 - val_binary_accuracy: 0.6760
Epoch 2/50
21/21 [==============================] - 6s 297ms/step - loss: 1.2258 - binary_accuracy: 0.7048 - val_loss: 1.2773 - val_binary_accuracy: 0.6829
Epoch 3/50
21/21 [==============================] - 6s 297ms/step - loss: 1.1378 - binary_accuracy: 0.7078 - val_loss: 1.1866 - val_binary_accuracy: 0.6568
Epoch 4/50
21/21 [==============================] - 6s 286ms/step - loss: 1.0694 - binary_accuracy: 0.7385 - val_loss: 1.1417 - val_binary_accuracy: 0.6794
Epoch 5/50
21/21 [==============================] - 6s 295ms/step - loss: 1.0049 - binary_accuracy: 0.7307 - val_loss: 1.1076 - val_binary_accuracy: 0.6568
Epoch 6/50
21/21 [==============================] - 6s 293ms/step - loss: 0.9555 - binary_accuracy: 0.7355 - val_loss: 1.0672 - val_binary_accuracy: 0.6672
Epoch 7/50
21/21 [==============================] - 6s 296ms/step - loss: 0

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1545 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 468 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)


22/22 [==============================] - 13s 544ms/step - loss: 0.6826 - binary_accuracy: 0.7725 - val_loss: 0.8332 - val_binary_accuracy: 0.6034
Epoch 2/50
22/22 [==============================] - 7s 297ms/step - loss: 0.6608 - binary_accuracy: 0.7834 - val_loss: 0.7762 - val_binary_accuracy: 0.6252
Epoch 3/50
22/22 [==============================] - 7s 300ms/step - loss: 0.6444 - binary_accuracy: 0.7981 - val_loss: 0.8024 - val_binary_accuracy: 0.6218
Epoch 4/50
22/22 [==============================] - 7s 296ms/step - loss: 0.6269 - binary_accuracy: 0.8018 - val_loss: 0.9981 - val_binary_accuracy: 0.4504
Epoch 5/50
22/22 [==============================] - 7s 302ms/step - loss: 0.6169 - binary_accuracy: 0.8081 - val_loss: 0.8148 - val_binary_accuracy: 0.5866
Epoch 6/50
22/22 [==============================] - 6s 287ms/step - loss: 0.6012 - binary_accuracy: 0.8115 - val_loss: 0.7841 - val_binary_accuracy: 0.6672
Epoch 7/50
22/22 [==============================] - 6s 294ms/step - loss: 

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1566 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 476 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)


21/21 [==============================] - 12s 503ms/step - loss: 0.5980 - binary_accuracy: 0.8016 - val_loss: 0.8173 - val_binary_accuracy: 0.6102
Epoch 2/50
21/21 [==============================] - 6s 277ms/step - loss: 0.5833 - binary_accuracy: 0.8104 - val_loss: 0.7246 - val_binary_accuracy: 0.6984
Epoch 3/50
21/21 [==============================] - 6s 277ms/step - loss: 0.5654 - binary_accuracy: 0.8197 - val_loss: 0.8512 - val_binary_accuracy: 0.5679
Epoch 4/50
21/21 [==============================] - 6s 279ms/step - loss: 0.5535 - binary_accuracy: 0.8373 - val_loss: 0.7204 - val_binary_accuracy: 0.6825
Epoch 5/50
21/21 [==============================] - 6s 276ms/step - loss: 0.5477 - binary_accuracy: 0.8219 - val_loss: 0.8574 - val_binary_accuracy: 0.6561
Epoch 6/50
21/21 [==============================] - 6s 280ms/step - loss: 0.5413 - binary_accuracy: 0.8320 - val_loss: 0.6984 - val_binary_accuracy: 0.6843
Epoch 7/50
21/21 [==============================] - 6s 283ms/step - loss: 

In [ ]:
# Store the data in X_test, y_test variables by iterating over the batches
batch_size =1
X_test, y_test = next(test_set)
for i in range(int(len(test_set)/batch_size)-1): #1st batch is already fetched before the for loop.
    img , label = next(test_set)
    X_test = np.append(X_test, img, axis=0 )
    y_test = np.append(y_test, label, axis=0)

print(X_test.shape, y_test.shape)
# train 
batch_size =1
X_train, y_train = next(training_set)
for i in range(int(len(training_set)/batch_size)-1): #1st batch is already fetched before the for loop.
    img , label = next(training_set)
    X_train = np.append(X_train, img, axis=0 )
    y_train = np.append(y_train, label, axis=0)

print(X_train.shape, y_train.shape)

(16, 224, 224, 3) (16, 7)
(324, 224, 224, 3) (324, 7)


In [ ]:
from sklearn.model_selection import KFold
i=1
df_metrics = pd.DataFrame()
kf = KFold(n_splits = 5, shuffle = True, random_state = None)
for train_index, test_index in kf.split(df):
    trainData = df.iloc[train_index]
    testData = df.iloc[test_index]
    print('Initializing Kfold %s'%str(i))
    print('Train shape:',trainData.shape)
    print('Test shape:',testData.shape)
    epochs = 30
    train_datagen = ImageDataGenerator(samplewise_center=True,rescale=1./255,validation_split=0.2)
    test_datagen = ImageDataGenerator(rescale=1. / 255) 

    training_set = train_datagen.flow_from_dataframe(dataframe=trainData,
                                                            directory="/content/principles-of-smile/K-fold/train",
                                                            x_col="img",
                                                            y_col=classes,
                                                            target_size=(224, 224),
                                                            batch_size=1,
                                                            class_mode="raw",seed=0)


    # valid_data_generation = ImageDataGenerator(samplewise_center=True,rescale = 2/255.)

    # valid_set = train_datagen.flow_from_dataframe(dataframe=trainData,
    #                                                         directory="/content/principles-of-smile/K-fold/train",
    #                                                         x_col="img",
    #                                                         y_col=classes,
    #                                                         subset="validation",
    #                                                         target_size=(224, 224),
    #                                                         batch_size=16,
    #                                                         class_mode="raw",seed=0)

    # preprocessing the testing set
    # test_data_generation = ImageDataGenerator(samplewise_center=True,rescale = 2/255.)

    test_set = test_datagen.flow_from_dataframe(dataframe=testData,
                                                            directory="/content/principles-of-smile/K-fold/test",
                                                            x_col="img",
                                                            y_col=classes,
                                                            target_size=(224, 224),
                                                            batch_size=1,
                                                            class_mode="raw",seed=0, shuffle = False)
    initial_learning_rate = 0.0002
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate),loss = 'BinaryCrossentropy',metrics=['binary_accuracy'])
    #fit model
    history=model.fit(x=training_set, validation_data=valid_set, epochs=10 ,callbacks=[es])
    i+=1



In [ ]:
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

i=1
# df_metrics = pd.DataFrame()
estimator = KerasClassifier(build_fn=MODEL, epochs=200, batch_size=5, verbose=0)
kf = KFold(n_splits = 5, shuffle = True, random_state = None)
results = cross_val_score(estimator,X_train, y_train, cv=kf)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
for train_index, test_index in kf.split(df):
    trainData = df.iloc[train_index]
    testData = df.iloc[test_index]
    print('Initializing Kfold %s'%str(i))
    print('Train shape:',trainData.shape)
    print('Test shape:',testData.shape)
    epochs = 30
    train_datagen = ImageDataGenerator(samplewise_center=True,rescale=1./255,validation_split=0.2)
    test_datagen = ImageDataGenerator(rescale=1. / 255) 

    training_set = train_datagen.flow_from_dataframe(dataframe=trainData,
                                                            directory="/content/principles-of-smile/K-fold/train",
                                                            x_col="img",
                                                            y_col=classes,
                                                            target_size=(224, 224),
                                                            batch_size=1,
                                                            class_mode="raw",seed=0)


    # valid_data_generation = ImageDataGenerator(samplewise_center=True,rescale = 2/255.)

    # valid_set = train_datagen.flow_from_dataframe(dataframe=trainData,
    #                                                         directory="/content/principles-of-smile/K-fold/train",
    #                                                         x_col="img",
    #                                                         y_col=classes,
    #                                                         subset="validation",
    #                                                         target_size=(224, 224),
    #                                                         batch_size=16,
    #                                                         class_mode="raw",seed=0)

    # preprocessing the testing set
    # test_data_generation = ImageDataGenerator(samplewise_center=True,rescale = 2/255.)

    test_set = test_datagen.flow_from_dataframe(dataframe=testData,
                                                            directory="/content/principles-of-smile/K-fold/test",
                                                            x_col="img",
                                                            y_col=classes,
                                                            target_size=(224, 224),
                                                            batch_size=1,
                                                            class_mode="raw",seed=0, shuffle = False)
    initial_learning_rate = 0.0002
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=initial_learning_rate),loss = 'BinaryCrossentropy',metrics=['binary_accuracy'])
    #fit model
    history=model.fit(x=training_set, validation_data=valid_set, epochs=10 ,callbacks=[es])
    i+=1



In [ ]:
pip install scikeras

  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.8.2:
      Successfully uninstalled importlib-metadata-4.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


In [ ]:
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

train_datagen = ImageDataGenerator(samplewise_center=True,rescale=2./255)
test_datagen = ImageDataGenerator(samplewise_center=True,rescale=2./255) 

training_set = train_datagen.flow_from_dataframe(dataframe=df,
                                                        directory="/content/principles-of-smile/K-fold/train",
                                                        x_col="img",
                                                        y_col=classes,
                                                        target_size=(224, 224),
                                                        batch_size=1,
                                                        class_mode="raw",seed=0)

test_set = test_datagen.flow_from_dataframe(dataframe=df,
                                                        directory="/content/principles-of-smile/K-fold/test",
                                                        x_col="img",
                                                        y_col=classes,
                                                        target_size=(224, 224),
                                                        batch_size=1,
                                                        class_mode="raw",seed=0, shuffle = False)

# Store the data in X_test, y_test variables by iterating over the batches
batch_size =1
X_test, y_test = next(test_set)
for i in range(int(len(test_set)/batch_size)-1): #1st batch is already fetched before the for loop.
    img , label = next(test_set)
    X_test = np.append(X_test, img, axis=0 )
    y_test = np.append(y_test, label, axis=0)

print(X_test.shape, y_test.shape)

# train 
batch_size =1
X_train, y_train = next(training_set)
for i in range(int(len(training_set)/batch_size)-1): #1st batch is already fetched before the for loop.
    img , label = next(training_set)
    X_train = np.append(X_train, img, axis=0 )
    y_train = np.append(y_train, label, axis=0)

print(X_train.shape, y_train.shape)



estimator = KerasClassifier(build_fn=MODEL, epochs=10, batch_size=5, verbose=0)
kf = KFold(n_splits = 5, shuffle = True, random_state = None)
results = cross_val_score(estimator,X_train, y_train, cv=kf)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1947 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2371 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 516 validated image filenames.
Found 92 validated image filenames.
(92, 224, 224, 3) (92, 7)
(516, 224, 224, 3) (516, 7)


/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will rai

Baseline: 13.38% (5.34%)


In [ ]:
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

train_datagen = ImageDataGenerator(samplewise_center=True,rescale=2./255)
test_datagen = ImageDataGenerator(samplewise_center=True,rescale=2./255) 

training_set = train_datagen.flow_from_dataframe(dataframe=df,
                                                        directory="/content/principles-of-smile/K-fold/train",
                                                        x_col="img",
                                                        y_col=classes,
                                                        target_size=(224, 224),
                                                        batch_size=1,
                                                        class_mode="raw",seed=0)

test_set = test_datagen.flow_from_dataframe(dataframe=df,
                                                        directory="/content/principles-of-smile/K-fold/test",
                                                        x_col="img",
                                                        y_col=classes,
                                                        target_size=(224, 224),
                                                        batch_size=1,
                                                        class_mode="raw",seed=0, shuffle = False)

# Store the data in X_test, y_test variables by iterating over the batches
batch_size =1
X_test, y_test = next(test_set)
for i in range(int(len(test_set)/batch_size)-1): #1st batch is already fetched before the for loop.
    img , label = next(test_set)
    X_test = np.append(X_test, img, axis=0 )
    y_test = np.append(y_test, label, axis=0)

print(X_test.shape, y_test.shape)

# train 
batch_size =1
X_train, y_train = next(training_set)
for i in range(int(len(training_set)/batch_size)-1): #1st batch is already fetched before the for loop.
    img , label = next(training_set)
    X_train = np.append(X_train, img, axis=0 )
    y_train = np.append(y_train, label, axis=0)

print(X_train.shape, y_train.shape)



estimator = KerasClassifier(build_fn=MODEL, epochs=20, batch_size=5, verbose=0)
kf = KFold(n_splits = 5, shuffle = True, random_state = None)
results = cross_val_score(estimator,X_train, y_train, cv=kf)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 1947 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2371 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 516 validated image filenames.
Found 92 validated image filenames.
(92, 224, 224, 3) (92, 7)
(516, 224, 224, 3) (516, 7)


/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  "``build_fn`` will be renamed to ``model`` in a future release,"
/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py:290: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will rai

Baseline: 16.47% (5.89%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
